### Packages

In [1]:
import urllib.request as urllib
from bs4 import BeautifulSoup
import re
import json

### Functions

In [2]:

# Read in a webpage
def read_in_page(url):
    #Query the website and return the html to the variable 'page'
    page = urllib.urlopen(url)
    soup = BeautifulSoup(page, "lxml")
    return(soup)


# Get all relevant links in a webpage
def get_links(page):
    links = page.find_all("a", class_="icon-arrow")
    links_new = []
    for link in links:
        sep = '?'
        rest = link['href'].split(sep, 1)[0]
        links_new.append(rest)
    return(links_new)


# Returns a dictionary {study-name: az-url}
def get_links_studies(page):
    links = page.find_all("a", class_="icon-arrow")
    links_new = {}
    for link in links:
        sep = '?'
        rest = link['href'].split(sep, 1)[0]
        # Get rid of everything before "student" and replace with https://
        temp = rest.split('student')
        rest = 'https://student' + temp[1]
        if rest.endswith('/'):
        # only do this if there isnt a slash at the end of rest
            links_new[link.string] = rest + "az"
        else:
            links_new[link.string] = rest + "/az"
    return(links_new)  


def get_studies_per_faculty(faculty_url):
    faculty_page = read_in_page(faculty_url)
    study_links = get_links_studies(faculty_page) # returns a dictionary {}
    return(study_links)


def get_az_links(az_url):
    az_page = read_in_page(az_url)
    az_links = get_links(az_page)
    az_links_new = []

    # get all the links on an AZ page
    for link in az_links:
        link = "http://www.student.uva.nl" + link
        az_links_new.append(link)
    return(az_links_new)


# Returns a list of keywords when given a question URL.
def get_keywords(question_url):
    # grab keywords with beautiful soup
    question_page = read_in_page(question_url)
    keyword_tag = question_page.findAll("meta", {"name": 'keywords'})
    keywords = keyword_tag[0]['content']
    return([keywords])

### Dictionary function

In [47]:
'''
def make_uva_dict():
    main_url = 'http://student.uva.nl/opleidingen/opleidingenlijst.html?'
    main_page = read_in_page(main_url)
    faculties = ['feb', 'fdr', 'fgw', 'medicine', 'fmg', 'fnwi']
    uva = {}

    for faculty in faculties:
        
        faculty_url = main_url + 't=' + faculty
        faculty_links = get_studies_per_faculty(faculty_url)
        uva[faculty] = faculty_links
        
        for study, az_link in uva[faculty].items():
            az_links_per_study = get_az_links(az_link)
            az_link_dict = {}

            for link in az_links_per_study:
                keywords = get_keywords(link)
                az_link_dict[link] = keywords
            uva[faculty][study] = az_link_dict
            
            print(study, "done")
        print(faculty, "done--------------------------------------------------------------------")

    return uva
'''

def make_faculty_dict(uva, faculty):
    main_url = 'http://student.uva.nl/opleidingen/opleidingenlijst.html?'
    main_page = read_in_page(main_url)

    uva = {}
    faculty_url = main_url + 't=' + faculty
    faculty_links = get_studies_per_faculty(faculty_url)
    uva[faculty] = faculty_links
    
    for study, az_link in uva[faculty].items():
        az_links_per_study = get_az_links(az_link)
        az_link_dict = {}
        
        for link in az_links_per_study:
            keywords = get_keywords(link)
            az_link_dict[link] = keywords
        uva[faculty][study] = az_link_dict
            
        print(study, "done")
    print(faculty, "done--------------------------------------------------------------------")

    return uva


In [49]:
uva_dict = {}
faculties = ['feb', 'fdr', 'fgw', 'medicine', 'fmg', 'fnwi']
i = 0

while i < len(faculties):
    try:
        uva_dict = make_faculty_dict(uva_dict, faculties[i])
    except ConnectionResetError:
        i -= 1
       
    i += 1


with open("uva_json_file", 'w') as f:
    json.dump(uva_dict, f)

Economics (Master's) done
Economics and business (bachelor) done
Economics and Business Economics (bachelor) done
Economie en Bedrijfskunde (bachelor) done
Business Administration (Master's) done
Fiscale economie (bachelor) done
Actuariële wetenschappen (bachelor) done
Econometrics (Master's) done
Business Administration (bachelor) done
Entrepreneurship (joint degree VU and UvA) done
Politics, Psychology, Law and Economics (bachelor) done
Actuarial Science and Mathematical Finance (Master's) done
Accountancy and Control (Master's) done
Econometrie en operationeel research (bachelor) done
Business Economics (Master's) done
Fiscale economie (master) done
feb done--------------------------------------------------------------------
Fiscaal recht (bachelor) done
Rechtsgeleerdheid (bachelor) done
Publiekrecht: Gezondheidsrecht (master) done
Publiekrecht: Militair recht (master) done
Politics, Psychology, Law and Economics (bachelor) done
Publiekrecht: Staats- en bestuursrecht (master) done
A

URLError: <urlopen error [Errno -3] Temporary failure in name resolution>

### Turns UvA dictionary into a JSON file

In [ ]:

with open("uva_json_file", 'w') as f:
    json.dump(uva_dict, f)
